<a href="https://colab.research.google.com/github/italofarve/projects/blob/main/natural_language_processing_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Procesamiento de lenguaje natural aplicado a la clasificación de mensajes de 7 categorías distintas. 


El modelo implementado fue un SVM con un 0.9849 de precisión. Se uso scikit-learn, alternativamente se podría usar las herramientas de Google o AWS. 


A continuación se explican:

- Los detalles del proyecto
- El desarrollo de la solución
- Se compara la performance del modelo SVM con otros modelos 


Contenido del proyecto


![Contenido del proyecto](https://github.com/italofarve/projects/blob/main/nlp-text-classification.png?raw=true)


# Detalles de la misión
Nuestro departamento de inteligencia ha clasificado todas las comunicaciones interceptadas
durante las últimas semanas en 7 categorías distintas, atendiendo a las divisiones Rebeldes a las
que pertenecen. Su misión es conseguir que el sistema de clasificación sea capaz de clasificar una
nueva transmisión de manera automática para que el departamento de operaciones pueda evaluar
las amenazas más rápido.

El resultado de esta misión deben ser 2 ejecutables: * train: Recibirá como único argumento el
nombre de una carpeta de entrada. Esta carpeta contendrá una subcarpeta por cada categoría,
dentro de las cuales estarán los ejemplos. A partir de esa carpeta, train entrenará un modelo y
generará un archivo "model" con el modelo entrenado. Ejemplo de uso:

# Pasos seguidos en la solución

## 1. Observación de los datos

En primer lugar, teniendo en cuenta que era un problema de clasificación de textos, se empezo a bucear dentro de las carpetas observando las características del contenido de los archivo de forma manual.

**Se observo:**

- Es un problema de clasificación similar al de clasificación de textos o  imágenes, cuyas muestras están ordenadas por carpetas (categorías). Tendremos en cuenta todos los caracteres por si los rebeldes enviaron mensajes secretos.

- Los archivos con un formato de nombre ".!31339!252524" estaban ocultos al explorador de archivos. Una manipulación manual de división de dataset podría provocar copiar aquellos archivos en los conjuntos de entrenamiento y test. Se verifico que los datos no se repitieran utilizando linea de comandos $ ls -l"


- Los archivos no contenian datos o metadatos como: from, subjetc, pero sí emails, firmas que de cierto modo representan las relaciones entre individuos y junto con el tipo de contenido que se comparte generán patrones que muestran una relación entre indivuos y/o fortalecen la categorización. Por ejemplo: escritores de contenido económico para diferentes medios pueden compartir estilo de prosa y colaborar entre ellos más que escritores de distintos tipos de contenido. Seguimos pensando en trabajar con todos los caracteres.


- Se observa documentos con distinta cantidad de caracteres por lo que necesitaremos de técnicas para compensarlo. Nos decidimos por usar Scikit-learn para empezar a manipular los datos por dos motivos: podemos aprovecharnos de su arquitectura y optimización de procesos (he seguido la misma estrategia para analizar imágenes, usando primero scikit-learn y luego añadiendo OpenCV. En este caso podría ser Scikit-learn + NLTK/TensorFlow o herramientas cloud de Google / AWS). 




## 2. Cargando datos para seguir analizandolos

- Previamente hicimos una carga de datos para visualizar todos datos en su conjunto, sin embargo aprovecharemos esta fase para seguir observando los datos.

- **división de datos:** Hemos extraido 19 elementos de cada categoría para que nos sirva de test en este primer ensayo. El 20 % de 400 es 20 y en cada categoria tenemos muestras que rondan los 400 - 600 muestras: 

--- carpeta de datos de entrenamiento: dataset

--- carpeta de datos de test: test


In [ ]:
# Importamos el modulo datasets para empezar a manipular los datos
import sklearn.datasets as skds

In [ ]:
# Carpeta de nuestros datos de entrenamiento 
path_train = 'dataset'

- load_files nos ayuda a cargar los datos de una carpeta (path_train) y carga los datos en memoria (load_content=True), barajamos los datos como antes de repartir barajas (shuffle=True) y añadimos una semilla para poder replicar el experimento (random_state=42).
- Es importante añadir un "encoding" de lo contrario se cargaran los archivos como "bytes" y no podremos utilizar otras funciones para procesar texto como las de extracción de características: CountVectorizer, TfidfTransformer, etc

In [ ]:
# cargamos los datos
files_train = skds.load_files(path_train, load_content=True, shuffle=True, encoding='latin1', random_state=42)

In [ ]:
# Contamos los datos que tenemos de cada categoria
import collections
print(collections.Counter(files_train.target))

Counter({4: 590, 5: 577, 1: 575, 0: 574, 6: 527, 3: 471, 2: 446})


In [ ]:
#Cantidad total de archivos para el entrenamiento
len(files_train.data)

3760

- Observamos que efectivamente hay un número diferente de archivos por cada categoría pero scikit-learn nos da funciones para poder compensar de este desbalance. Lo veremos más adelante.

- Apuntamos el orden de las categorias, nos servirá para crear el archivo "classify.py". De esta forma aprovecharemos la estructura de carpetas que tiene scikit-learn.

In [ ]:
files_train.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [ ]:
# Visualizamos orden de categorias
files_train.target_names

['exploration',
 'headhunters',
 'intelligence',
 'logistics',
 'politics',
 'transportation',
 'weapons']

- Con las siguientes dos instrucciones podemos visualizar el contenido de un archivo y su categoría. Observamos que un ser humano incorporado al sistema de retroalimentación del modelo sería de mucha utilidad.

In [ ]:
# Visualizando contenidos de distintos archivos cambiando ".data[numero]."
# Observamos que hablan de como conducen los autralianos y podemos deducir la categoría 
# "transportation"

print("\n".join(files_train.data[9].split("\n")[:12]))

Article-I.D.: cactus.1993Apr6.060553.22453

In article <YfkBJQS00Uh_E9TFo_@andrew.cmu.edu> "Daniel U. Holbrook" <dh3q+@andrew.cmu.edu> writes:
>>>
>
[stuff about RHD deSoto's deleted]

>Well Sweden and Australia, and lord knows wherever else used to drive on
Australians still do drive on the "wrong" side of the road. I believe
Sweden changed in 1968. The way I heard it was that they swapped
all the traffic signs around one Sunday....



In [ ]:
print(files_train.target_names[files_train.target[9]])

transportation


### 3. Extracción de características de los archivos de texto

CountVectorizer incluye el preprocesamiento del texto, la tokenización y el filtrado de las palabras, como resultado construye un diccionario de caracteristicas y los documentos quedan convertidos en vectores de caracteristicas con los que ya podemos trabajar.

In [ ]:

from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(files_train.data)
X_train_counts.shape

(3760, 46585)

In [ ]:
# Podemos saber las veces que aparece la palabra algorithm en nuestro vocabulario.
count_vect.vocabulary_.get(u'algorithm')

7779

**¿Por qué es importante contar cuantas veces aparece una palabra?**

Es un primer paso hacia la predicción. Si aparece mucho la palabra "conducir" y "viajes", probablemente el texto encaje en la categoría "transportation"

**Limites del conteo de palabras**

La frecuencia que aparece una palabra puede ser mayor en textos más grandes y menor en textos más cortos aunque se refieran al mismo tema, por lo que observamos un problema. 

Para resolver esto se resuelve dividiendo el número de veces que aparece una palabra en un documento por el número total de palabras del documento **(Term Frequencies)**. Adicionalmente se aplica una reducción de los pesos de las palabras que aparecen en muchos documentos que son menos informativas **(“Term Frequency times Inverse Document Frequency”)**


Aplicando ambas transformaciones: Term Frequencies y Term Frequency times Inverse Document Frequency (**TfidfTransformer**)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(3760, 46585)

## 4. Probando clasificador Naive Bayes

Empezamos probando el Naive Bayes. Observar que tenemos que usar el mismo proceso de transformación de texto realizado con los datos de entrenamiento.

El Naive Bayes Multinomial es una de las variantes usadas en clasificación de texto. La distrubución está parametrizada por vectores para cada clase y se trabaja con la probabilidad de que una caracteristica "i" aparezca en una muestra perteneciente a una clase "y". Por defecto el suavizado ("alfa" = 1) que también se conoce como Laplace smoothing.

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, files_train.target)

In [ ]:
docs_new = ["one-day Navy Scientific Visualization and Virtual Reality Seminar. The purpose of the seminar is to present and exchange information for Navy-related scientific visualization and virtual reality programs, research, developments, and applications.", 
            ": Colin Greenwood from Scotland Yard did a study that showed that gun : control has had no effect on crime or murder rates in the UK.  His book,: _Firearms_Controls_, has been published in London by Keegan Paul (name : may be misspelled)."]

X_new_counts = count_vect.transform(docs_new)

# Entrenamos el modelo
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
            print('%r => %s' % (doc, files_train.target_names[category]))
            
           

'one-day Navy Scientific Visualization and Virtual Reality Seminar. The purpose of the seminar is to present and exchange information for Navy-related scientific visualization and virtual reality programs, research, developments, and applications.' => headhunters
': Colin Greenwood from Scotland Yard did a study that showed that gun : control has had no effect on crime or murder rates in the UK.  His book,: _Firearms_Controls_, has been published in London by Keegan Paul (name : may be misspelled).' => weapons


## 5. ¿Por qué construir un PipeLine?


Hasta ahora hemos aplicado el vectorizado, y el tratamiento de ocurrencias a frecuencias tanto para los datos de entrenamiento como para test. Sin embargo Scikit-learn nos permite automatizar todos esas fases en una sentencia de código. Esto nos permitirá probar nuevos módelos de forma más ágil y es lo que usaremos en el archivo "train.py" para crear nuestro modelo de clasificación.

In [ ]:
## Construyendo un pipeline de vectorizado, tf-idf y empleando el Naive Bayes Multinomial


from sklearn.pipeline import Pipeline
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

In [ ]:
# Entrenamos el modelo
text_clf.fit(files_train.data, files_train.target)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

## ¿Dónde están nuestras muestras de test?

Hemos guardado los archivos de test en la carpeta llamada "test". Usar la misma estrategia y estructura que nos proporciona scikit-learn para cargar datos nos proporcionará agilidad para probar "n" modelos diferentes. Ver el notebook adjuntado llamado "otros-modelos.ypnb"

In [ ]:
path_test = 'test'

In [ ]:
# files_test = skds.load_files(path_test,load_content=False)
files_test = skds.load_files(path_test, load_content=True, shuffle=True, encoding='latin1', random_state=42)
files_test.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [ ]:
# Observamos que 133 = 19 * 7, por lo que no se nos ha colando archivos ocultos.

len(files_test.filenames)
# files_test.keys()
# files_test.filenames

133

## 6. Evaluamos performance del clasificador Naive Bayes Multinomial


### Precisión

In [ ]:
# Accuracy de 92.48%

import numpy as np
docs_test = files_test.data
predicted = text_clf.predict(docs_test)
np.mean(predicted == files_test.target)

0.924812030075188

### Reporte más completo del modelo

Recordemos que el recall es la tasa de verdaderos positivos, muestras que han sido correctamente identificadas

Observemos el **recall** para la categoría **intelligence** es de 0.53. Del reporte de métricas y la matriz de confusión vemos que el clasificador está confundiendo casi la mitad de muestras con la categoría **weapons**


In [ ]:
from sklearn import metrics
print(metrics.classification_report(
    files_test.target, 
    predicted,
    target_names=files_test.target_names))

                precision    recall  f1-score   support

   exploration       1.00      1.00      1.00        19
   headhunters       1.00      0.95      0.97        19
  intelligence       1.00      0.53      0.69        19
     logistics       1.00      1.00      1.00        19
      politics       0.95      1.00      0.97        19
transportation       1.00      1.00      1.00        19
       weapons       0.68      1.00      0.81        19

      accuracy                           0.92       133
     macro avg       0.95      0.92      0.92       133
  weighted avg       0.95      0.92      0.92       133



In [ ]:
metrics.confusion_matrix(files_test.target, predicted)

array([[19,  0,  0,  0,  0,  0,  0],
       [ 0, 18,  0,  0,  1,  0,  0],
       [ 0,  0, 10,  0,  0,  0,  9],
       [ 0,  0,  0, 19,  0,  0,  0],
       [ 0,  0,  0,  0, 19,  0,  0],
       [ 0,  0,  0,  0,  0, 19,  0],
       [ 0,  0,  0,  0,  0,  0, 19]])

## 7. Probando el clasificador SVM

Una de las ventajas de emplear un SVM es que son eficases cuando el número de dimensiones de las muestras es muy grande (recordemos que nuestras muestras han sido vectorizadas). Por ejemplo SVM para dos clases trabaja apoyandose en dos vectores de soporte, uno por cada clase, que marcan las fronteras de clasificación. Imaginemos la frontera de dos paises con una zona neutral de igual distancia para ambas partes. Los comienzos de cada zona neutral estarán marcados por lineas o hiperplanos, la distancia perpendicular entre ambas lineas se conoce como margen, este margen podría ser máximo o minimo. Las funciones de perdida se pueden considerar como un limite superior del error de clasificación.

- loss (hace referencia a soft-margin)
- penalty (hace referecia al entorno de regulación)
- alpha (constante, cuanto más alto es el valor, más fuerte es la regularización)
- max_iter (número de épocas)

In [ ]:
# Usando SVM
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-3, random_state=42,
                          max_iter=5, tol=None)),
])

text_clf.fit(files_train.data, files_train.target)


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 SGDClassifier(alpha=0.001, max_iter=5, random_state=42,
                               tol=None))])

## 8. Evaluamos el performance del nuevo modelo SVM

### Precisión

In [ ]:
## Empleando clasificador
predicted = text_clf.predict(docs_test)

# calculando precisión
np.mean(predicted == files_test.target)

0.9849624060150376

### Reporte más completo

Recordemos que el recall es la tasa de verdaderos positivos, muestra que han sido correctamente identificadas

Observemos el **recall** para la categoría **intelligence** pasa de 0.53 a 0.95. Del reporte de métricas y la matriz de confusión vemos que el clasificador está confundiendo sólo 2 muestras. Este modelo es más optimo y procederemos a implementarlo para la entrega.


In [ ]:
from sklearn import metrics
print(metrics.classification_report(
    files_test.target, 
    predicted,
    target_names=files_test.target_names))

                precision    recall  f1-score   support

   exploration       1.00      1.00      1.00        19
   headhunters       0.95      1.00      0.97        19
  intelligence       1.00      0.95      0.97        19
     logistics       1.00      1.00      1.00        19
      politics       1.00      1.00      1.00        19
transportation       0.95      1.00      0.97        19
       weapons       1.00      0.95      0.97        19

      accuracy                           0.98       133
     macro avg       0.99      0.98      0.98       133
  weighted avg       0.99      0.98      0.98       133



In [ ]:
metrics.confusion_matrix(files_test.target, predicted)

array([[19,  0,  0,  0,  0,  0,  0],
       [ 0, 19,  0,  0,  0,  0,  0],
       [ 0,  0, 18,  0,  0,  1,  0],
       [ 0,  0,  0, 19,  0,  0,  0],
       [ 0,  0,  0,  0, 19,  0,  0],
       [ 0,  0,  0,  0,  0, 19,  0],
       [ 0,  1,  0,  0,  0,  0, 18]])

## 9. Conclusiones y recomendaciones

- Mantenemos las 19 muestras para test por cada categoría y el analisis de todos los caracteres. El proceso de extracción de características nos permite ganar ágilidad pero si la cantidad de datos de entrenamiento se multiplica habría que trabajar en la eliminación de algunas palabras.

- Para implementar elegimos el modelo basado en SVM con un 0.9849 de precisión despues de hacer pruebas con otros algoritmos que no superaban el perfomance del SVM (ver el notebook "otros-modelos.ipynb").

- De las diferencias en las matrices de confusión concluimos la necesidad de utilizar, adicionalmente, alguno de los algoritmos que superaban el 98% de precisión para una próxima implementación o implementación A/B, esto nos permitirá balancear los errores de clasificación y detectar cambios en las transmisiones.

- También recomendamos establecer una sistema de votaciones con 5 algoritmos de precisión mayor de 95%, estableciendo un threshold de probabilidad que permita incluir la intervención de un humano dentro del ciclo de mejora del sistema de clasificación de textos. Las costumbres y conexiones humanas cambian, eso se podría reflejar en los documentos, y el sistema debería adaptarse de manera continua. 

- Scikit-learn nos permite guardar modelos haciendo uso de la liberia "joblib" y es la que hemos utilizado al no poder crear archivos ejecutables. Recomendamos su uso ya que su peso es muchisimo más liviano y son más seguros. Un archivo ejecutable siempre podría contener codigo malicioso.


